In [2]:
# setting up dependencies
import numpy as np
import pandas as pd
from splinter import Browser
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import requests as req

In [3]:
# setting up to scrape the NASA website. Assigning the variables for reference later.

# empty dictionary and empty list for future use
news_data = {}
paragraph_text = []

In [4]:
# base url for the target site
base_url = "https://mars.nasa.gov/" #anything special for https??

#starting scrape here
nasa_url = "https://mars.nasa.gov/news/"

#first response URL here...think so?
response_1 = req.get(nasa_url)

# ahh beautiful soup; running everything through its easy button
nasa_soup = bs(response_1.text, 'html.parser')

In [5]:
# not making this easy - searching for what we need

#looking for classes
soup_div = nasa_soup.find(class_="slide")

#looking for anchors
soup_news = soup_div.find_all('a')

#pulling and cleaning
news_title = soup_news[1].get_text().strip()

In [7]:
# finding paragraphs, connecting, gets next response and then sends to bs
soup_p = soup_div.find_all('a', href=True)
soup_p_url = soup_p[0]['href']
paragraph_url = base_url + soup_p_url
response_2 = req.get(paragraph_url)                                          
para_soup = bs(response_2.text, "html.parser")                               
ww_paragraphs = para_soup.find(class_='wysiwyg_content')                     
paragraphs = ww_paragraphs.find_all('p') 

In [8]:
# cleaning paragraphs and adding to the list
for paragraph in paragraphs:                                                 
    clean_paragraph = paragraph.get_text().strip()                               
    paragraph_text.append(clean_paragraph)      

In [9]:
# adding items to the dictionary
news_data["news_title"] = news_title                                        
news_data["paragraph_text_1"] = paragraph_text[0]
news_data["paragraph_text_2"] = paragraph_text[1] 
news_data

{'news_title': 'Opportunity Hunkers Down During Dust Storm',
 'paragraph_text_1': 'NASA Mars Exploration Rover Status Report',
 'paragraph_text_2': 'Updated at 2:25 p.m. PDT on July 26, 2018'}

In [10]:
# Using splinter to navigate the site and find the image url for the current Featured Mars Image and assign the url string to a variable called featured_image_url.
browser = Browser('chrome', headless=False)                                  
jpl_fullsize_url = 'https://photojournal.jpl.nasa.gov/jpeg/'                 
jpl_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"



In [11]:
# visits search URL with automated browser
# acquires response from URL
# sends response to beautiful soup
browser.visit(jpl_url)                                                       
jpl_html = browser.html                                                      
jpl_soup = bs(jpl_html, 'html.parser')                                       

In [12]:
#extracting all the images and putting them / appending them to the list
featured_image_list = []                                                     

for image in jpl_soup.find_all('div',class_="img"):                          
    featured_image_list.append(image.find('img').get('src')) 

In [13]:
# extracts first image found and cleaning them up
feature_image = featured_image_list[0]                                       
temp_list_1 = feature_image.split('-')                                       
temp_list_2 = temp_list_1[0].split('/')                                      
featured_image_url = jpl_fullsize_url + temp_list_2[-1] + '.jpg' 

In [14]:
featured_image_url
browser.quit() 

In [15]:
# getting the weather data and twitter data
browser = Browser('chrome', headless=False)                                  
tweet_url = 'https://twitter.com/marswxreport?lang=en'                       
browser.visit(tweet_url)  
tweet_html = browser.html                                                   
tweet_soup = bs(tweet_html, 'html.parser') 
weather_info_list = []    
for weather_info in tweet_soup.find_all('p',class_="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text"):
    weather_info_list.append(weather_info.text.strip()) 

In [16]:
# isolates weather tweet
for value in reversed(weather_info_list):                                    
    if value[:3]=='Sol':                                                     
        mars_weather = value

In [17]:
#did it work?
mars_weather                                                                

'Sol 2171 (2018-09-14), high -12C/10F, low -65C/-84F, pressure at 8.79 hPa, daylight 05:43-17:59'

In [18]:
#cleaning up by closing the automated browser
browser.quit()  

In [19]:
#Visit the Mars Facts webpage here and use Pandas to scrape the table containing facts 
#about the planet including Diameter, Mass, etc.
#defining, extracting, and converting to a dataframe for eaiser use
facts_url = 'https://space-facts.com/mars/' 
fact_list = pd.read_html(facts_url)  
facts_df = fact_list[0]   

# converts dataframe to html table
facts_table = facts_df.to_html(header=False, index=False)                    
print(facts_table) 

<table border="1" class="dataframe">
  <tbody>
    <tr>
      <td>Equatorial Diameter:</td>
      <td>6,792 km</td>
    </tr>
    <tr>
      <td>Polar Diameter:</td>
      <td>6,752 km</td>
    </tr>
    <tr>
      <td>Mass:</td>
      <td>6.42 x 10^23 kg (10.7% Earth)</td>
    </tr>
    <tr>
      <td>Moons:</td>
      <td>2 (Phobos &amp; Deimos)</td>
    </tr>
    <tr>
      <td>Orbit Distance:</td>
      <td>227,943,824 km (1.52 AU)</td>
    </tr>
    <tr>
      <td>Orbit Period:</td>
      <td>687 days (1.9 years)</td>
    </tr>
    <tr>
      <td>Surface Temperature:</td>
      <td>-153 to 20 °C</td>
    </tr>
    <tr>
      <td>First Record:</td>
      <td>2nd millennium BC</td>
    </tr>
    <tr>
      <td>Recorded By:</td>
      <td>Egyptian astronomers</td>
    </tr>
  </tbody>
</table>


In [20]:
#getting high resolution images for each of Mars' hemispheres

#define browser again.
browser = Browser('chrome', headless=False)                                                                  

# defines search URL
usgs_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(usgs_url)
usgs_html = browser.html                                                     
usgs_soup = bs(usgs_html, 'html.parser')  
hemisphere_image_urls = []                                             

products = usgs_soup.find('div', class_='result-list')                       
hemispheres = products.find_all('div', class_='item')                        

for hemisphere in hemispheres:                                             
    title = hemisphere.find('div', class_='description')
    
    title_text = title.a.text                                                
    title_text = title_text.replace(' Enhanced', '')
    browser.click_link_by_partial_text(title_text)                           
    
    usgs_html = browser.html                                                 
    usgs_soup = bs(usgs_html, 'html.parser')                                 
    
    image = usgs_soup.find('div', class_='downloads').find('ul').find('li')  
    img_url = image.a['href']
    
    hemisphere_image_urls.append({'title': title_text, 'img_url': img_url})    
    
    browser.click_link_by_partial_text('Back') 

In [21]:
#all my image dictionaries
browser.quit()  
hemisphere_image_urls


[{'title': 'Cerberus Hemisphere',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]